In [ ]:
!pip install jaydebeapi
!pip install jpype1


In [ ]:
!echo %JAVA_HOME%

In [ ]:
import os
import jaydebeapi
import jpype
import dateutil.parser

import pandas as pd
from pandas.io.json import json_normalize

BASE_DIR = 'C:/Tools/serpro/'

class DassDB:

    conn = None
    url = None
    TEIID_JAR = 'jboss-dv-6.3.0-teiid-jdbc.jar'

    def __init__(self, url):
        self.url = url
        self.connect()

    def __del__(self):
        try:
            if self.conn:
                self.conn.close()
        except Exception as e:
            logger.error("Erro fechar a conexão DAAS")
            logger.exception(e)
            pass

    def connect(self):
        jar_path = os.path.join(BASE_DIR, DassDB.TEIID_JAR)
        if not jpype.isJVMStarted():
            # NOTE: after this PR is closed: https://github.com/baztian/jaydebeapi/pull/116
            #      we can upgrade JayDeBeApi and remove this code
            args = []
            class_path = [jar_path]
            class_path.extend(jaydebeapi._get_classpath())
            args.append('-Djava.class.path=%s' % os.path.pathsep.join(class_path))
            args.append('-Djavax.net.ssl.trustStore=%s' % os.path.join(BASE_DIR, 'daas.serpro.gov.br.jks'))
            jvm_path = jpype.getDefaultJVMPath()
            jpype.startJVM(jvm_path, *args)

        self.conn = jaydebeapi.connect("org.teiid.jdbc.TeiidDriver",
                                  self.url,
                                  [DAAS_DB_USER, DAAS_DB_PASS],
                                  jar_path)

    def query(self, sql):
        try:
            cursor = self.conn.cursor()
            cursor.execute(sql)
        except Exception as e:
            logger.exception(e)
            logger.error("Erro ao executar no DAAS: {}.".format(sql))
            self.connect()
            cursor = self.conn.cursor()
            cursor.execute(sql)
        desc = cursor.description
        registros = [dict(zip([str(col[0]) for col in desc], row)) for row in cursor.fetchall()]
        cursor.close()
        return registros    

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path  
from scipy.stats import uniform
from scipy.stats import kstest
from scipy.stats import lognorm

#SCREENS
def ks_test_of_function(array_data, function):
    '''Calculate the function (uniform or lognorm) of array_data and then calculate Kolmogorov–Smirnov test'''

    if function is 'uniform':
        loc, scale = uniform.fit(array_data)
        n = uniform(loc=loc, scale=scale)
    elif function is 'lognorm':
        s, loc, scale = lognorm.fit(array_data)
        n = lognorm(s=s, loc=loc, scale=scale)
    else: 
        return None
    return kstest(array_data, n.cdf)[0]

def calculate_screen_variables(df, screens, decimals=4):
    ''' Calculate the defined screens variables of the dataframe (columns are Tender and Bid_value). By default, 4 decimals of accuracy '''

    if 'CV' in screens:
        # Calculate the coefficient of variation (CV)
        mean_bid_value_by_tender = df.groupby(['Tender'])['Bid_value'].mean()
        std_by_tender = df.groupby(['Tender'])['Bid_value'].std()
        cv = pd.Series(std_by_tender / mean_bid_value_by_tender, name='CV').round(decimals=decimals)
        res = df.merge(cv, how='inner', left_on='Tender', right_on='Tender', sort=False)
        res.set_index(df.index,inplace=True)
        df = res
    if 'SPD' in screens:
        # Calculate the spread (SPD)
        max_bid_value_by_tender = df.groupby(['Tender'])['Bid_value'].max()
        min_bid_value_by_tender = df.groupby(['Tender'])['Bid_value'].min()
        spd = pd.Series((max_bid_value_by_tender - min_bid_value_by_tender) / min_bid_value_by_tender, name='SPD').round(decimals=decimals)
        res = df.merge(spd, how='inner', left_on='Tender', right_on='Tender', sort=False)
        res.set_index(df.index,inplace=True)
        df = res
    if 'DIFFP' in screens:    
        # Calculate the differences between the two lowest bids is the percentage difference (DIFFP)
        min_bid_value_by_tender = df.groupby(['Tender'])['Bid_value'].min()
        df_without_duplicates = df.drop_duplicates(subset=['Bid_value'])
        min2_bid_value_by_tender = df_without_duplicates.groupby(['Tender'])['Bid_value'].nsmallest(2).groupby(['Tender']).last()
        diffp = pd.Series((min2_bid_value_by_tender - min_bid_value_by_tender) / min_bid_value_by_tender, name='DIFFP').round(decimals=decimals)
        res = df.merge(diffp, how='inner', left_on='Tender', right_on='Tender', sort=False)
        res.set_index(df.index,inplace=True)
        df = res
    if 'RD' in screens:
        min_bid_value_by_tender = df.groupby(['Tender'])['Bid_value'].min()
        std_by_tender = df.groupby(['Tender'])['Bid_value'].std()
        df_without_duplicates = df.drop_duplicates(subset=['Bid_value'])
        min2_bid_value_by_tender = df_without_duplicates.groupby(['Tender'])['Bid_value'].nsmallest(2).groupby(['Tender']).last()
        df_max_bid_by_tenders = df.groupby(['Tender'])['Bid_value'].transform('max')
        df_losing_bids = df[~(df['Bid_value'] == df_max_bid_by_tenders)]
        std_losing_bids_by_tender = df_losing_bids.groupby(['Tender'])['Bid_value'].std()
        df_std_losing_bids_by_tender = pd.DataFrame({'Tender': std_losing_bids_by_tender.index, 'STD': std_losing_bids_by_tender.values})
        df_std_by_tender = pd.DataFrame({'Tender': std_by_tender.index, 'STD': std_by_tender.values})
        #df_std_losing_bids_by_tender['STD'] = df_std_losing_bids_by_tender['STD'].replace(0, np.nan)
        df_std_losing_bids_by_tender['STD'] = df_std_losing_bids_by_tender['STD'].fillna(df_std_by_tender['STD'])
        std_losing_bids_by_tender =  pd.Series(data=df_std_losing_bids_by_tender['STD'])
        rd = pd.Series((min2_bid_value_by_tender.reset_index(drop=True) - min_bid_value_by_tender.reset_index(drop=True)) / std_losing_bids_by_tender.reset_index(drop=True), name='RD')
        rd = pd.DataFrame({'Tender': min_bid_value_by_tender.index, 'RD': rd.values}).round(decimals=decimals)
        res = df.merge(rd, how='inner', left_on='Tender', right_on='Tender', sort=False)
        res.set_index(df.index,inplace=True)
        df = res
    if 'KURT' in screens:
        # Calculate Kurtosis statistic (KURTO)
        kurtosis_by_tender = df.groupby(['Tender'])['Bid_value'].apply(pd.DataFrame.kurt)
        kurtosis_by_tender = pd.Series(kurtosis_by_tender, name='KURT')
        kurtosis_by_tender = kurtosis_by_tender.fillna(0).round(decimals=decimals)
        res = df.merge(kurtosis_by_tender, how='inner', left_on='Tender', right_on='Tender', sort=False) 
        res.set_index(df.index,inplace=True)
        df = res
    if 'SKEW' in screens:    
        # Calculate Kewness statistic (SKEW)
        skew_by_tender = df.groupby(['Tender'])['Bid_value'].skew()
        skew_by_tender = pd.Series(skew_by_tender, name='SKEW')
        skew_by_tender = skew_by_tender.fillna(0).round(decimals=decimals)
        res = df.merge(skew_by_tender, how='inner', left_on='Tender', right_on='Tender', sort=False)
        res.set_index(df.index,inplace=True)
        df = res
    if 'KSTEST' in screens:
        # Calculate Kolmogorov-Smirnov statistic (KSTEST) for verifying if the bids in a tender follow a distribution (uniform, lognorm, etc...)
        kolmogorov_smirnov_by_tender = df.groupby(['Tender'])['Bid_value'].apply(lambda x: ks_test_of_function(x, 'uniform'))
        kolmogorov_smirnov_by_tender = pd.Series(kolmogorov_smirnov_by_tender, name='KSTEST').round(decimals=decimals)
        res = df.merge(kolmogorov_smirnov_by_tender, how='inner', left_on='Tender', right_on='Tender', sort=False)
        res.set_index(df.index,inplace=True)
        df = res
    if 'KSTEST_L' in screens:
        # Calculate Kolmogorov-Smirnov statistic (KSTEST) for verifying if the bids in a tender follow a distribution (uniform, lognorm, etc...)
        kolmogorov_smirnov_by_tender = df.groupby(['Tender'])['Bid_value'].apply(lambda x: ks_test_of_function(x, 'lognorm'))
        kolmogorov_smirnov_by_tender = pd.Series(kolmogorov_smirnov_by_tender, name='KSTEST_L').round(decimals=decimals)
        res = df.merge(kolmogorov_smirnov_by_tender, how='inner', left_on='Tender', right_on='Tender', sort=False)
        res.set_index(df.index,inplace=True)
        df = res
    return df

In [ ]:
from pathlib import Path  

DAAS_COMPRASNET_DB_HOST = '' #Endereço DAAS do Comprasnet
DAAS_SIASGNET_DB_HOST = '' #Endereço DAAS do SIASGNET
DAAS_DB_USER = '' #Usuário DAAS
DAAS_DB_PASS = '' #Senha DAAS
db_comprasnet = DassDB(DAAS_COMPRASNET_DB_HOST)
db_siasgnet = DassDB(DAAS_SIASGNET_DB_HOST)

#Dataframe com todos os itens das licitações
dfi = None
#Arquivos csv com o dataframe
nome_arquivo = 'C:/tmp/brazilian_comprasnet.csv'

punidos = ['02293852000140', '01785999000194', '96355946000140', '35596501000167', '03187742000166', '62436282000121', '88309620000158', '01478038000137', 
           '07386939000185', '11572018000184', '04717562000101', '04809827000100', '06332079000134', '25181298000104', '01140694000125', '08859696000118', 
           '37517158000143', '03737267000154', '13385812000144', '02845074000154', '49254634000160', '44164606000138', '14744743000180', '02332985000188']

if os.path.isfile(nome_arquivo):
    dfi = pd.read_csv(nome_arquivo, index_col='Bid');
    #dfi = pd.read_csv(nome_arquivo);
else:    
    #QTD_ADQUIRIDA
    itens = db_comprasnet.query("SELECT p.prgCod, p.numprp, p.coduasg, pri.ipgCod, l.lanCod, pro.prpCNPJ, l.lanValor, " +
                                " (case when l.lanValor = pri.ipgValorMinClassif then 1 else 0 end) as Winner, l.lanData, " + 
                                " pri.ipgValorRef, (pri.ipgValorRef - l.lanValor) as Difference_Bid_PTE, " +
                                " pri.ipgValorMinClassif, pri.ipgQuantidade, pri.prgCod, pri.ipgItem, pri.codmat " +
                                " FROM Comprasnet_VBL.tbl_Pregao p " +
                                " INNER JOIN Comprasnet_VBL.tbl_pregaoitem pri ON p.prgCod = pri.prgCod " +
                                " INNER JOIN Comprasnet_VBL.tbl_Lances l ON pri.ipgCod = l.ipgCod " +
                                " INNER JOIN Comprasnet_VBL.tbl_Proposta pro ON pro.prgCod = pri.prgCod AND pro.Cliente_ID = l.cliente_id " +                                
                                " WHERE pri.prgCod in (SELECT distinct proi.prgCod FROM Comprasnet_VBL.tbl_Proposta proi " +
                                " WHERE proi.prpCNPJ in ('02293852000140', '01785999000194', '96355946000140', '35596501000167', '03187742000166', '62436282000121', '88309620000158', " +
                                " '01478038000137', '07386939000185', '11572018000184', '04717562000101', '04809827000100', '06332079000134', '25181298000104', '01140694000125', " +
                                " '08859696000118', '37517158000143', '03737267000154', '13385812000144', '02845074000154', '49254634000160', '44164606000138', '14744743000180', " +
                                " '02332985000188')) " +
                                " and ipgQuantidade > 0 " +
                                " and l.lanStatus = 'V' " +                                
                                " and pri.ipgFormaJulg = 'V' " +
                                " and pri.ipgValorMinClassif is not null " +
                                " order by pri.ipgCod, pri.ipgItem, l.lanValor ")
    #print('itens:')
    #print(itens)

    #forçando conversão de campos texto
    for item in itens:
        #print('Item {}: {} - (Valor: {} - Min: {})'.format(item['ipgCod'],item['ipgItem'],item['lanValor'],item['ipgValorMinClassif']))
        item['prpCNPJ'] = str(item['prpCNPJ'])
        item['lanData'] = int(dateutil.parser.parse(item['lanData'], yearfirst=True).timestamp())
        item['lanValor'] = round(item['lanValor'],4)
        item['ipgValorMinClassif'] = round(item['ipgValorMinClassif'],4)
        item['Difference_Bid_PTE'] = round(item['Difference_Bid_PTE'],4)
    
    dfi = json_normalize(itens)
    dfi = dfi.rename(columns={'ipgCod': 'Tender', 
                             'lanCod': 'Bid', 
                             'prpCNPJ': 'Competitor', 
                             'lanValor': 'Bid_value', 
                             'lanData': 'Date', 
                             'ipgValorRef': 'Pre_Tender'})
    dfi.set_index('Bid', inplace=True)
    
    dfi['atualizado'] = 0
    dfi['Number_bids'] = 0
    dfi['especificidade'] = 0
    dfi['frequencia'] = 0
    dfi['material'] = ' '
    dfi['Collusive_competitor_original'] = 0
    dfi['Collusive_competitor'] = 0

    filepath = Path(nome_arquivo)  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    dfi.to_csv(filepath) 

prgCod = None
lances          = []
especificidades = []
frequencias     = []
materiais       = []
colusivo = 0
licitacoes = 0

for index, item in dfi.iterrows():
    #print('Item {}: {} - ({}){}'.format(item['ipgCod'],item['ipgQuantidade'],item['prpCNPJ'],item['prpRazaoSocial']))

    #Salta os registros atualizados
    if item['atualizado'] == 1:
        continue
        
    #encontrou novo pregão
    if item['prgCod'] != prgCod:
        prgCod = item['prgCod']
        print('LICITACAO: {}'.format(prgCod))

        #QTD_LANCES
        lances = db_comprasnet.query("SELECT pri.ipgCod, count(l.ipgCod) as qtdeLances " +
                                     " FROM Comprasnet_VBL.tbl_pregaoitem pri " +
                                     " INNER JOIN Comprasnet_VBL.tbl_Lances l ON pri.ipgCod = l.ipgCod " +
                                    " WHERE pri.prgCod = " + str(prgCod) +
                                    " AND l.lanStatus = 'V' " +
                                    " GROUP BY pri.ipgCod ")
        #print('lances:')
        #print(lances)


        #ESPECIFICIDADE   
        especificidades = db_comprasnet.query("SELECT pri.codmat, count(1) as especificidade " +
                                     " FROM Comprasnet_VBL.tbl_pregaoitem pri " +
                                     " INNER JOIN Comprasnet_VBL.tbl_pregaoitem prif ON pri.codmat = prif.codmat and prif.ipgCod != pri.ipgCod " +
                                    " WHERE pri.prgCod = " + str(prgCod) +
                                    " GROUP BY pri.codmat ")
        #print('especificidades:')
        #print(especificidades)


        #FREQUENCIA
        frequencias = db_comprasnet.query("SELECT prif.codmat, count(1) as frequencia " +
                                     " FROM Comprasnet_VBL.tbl_pregaoitem pri " +
                                     " INNER JOIN Comprasnet_VBL.tbl_propostaitem ppi ON pri.ipgCod = ppi.ipgCod and ppi.ippIndAdjudicado = 'S' " +
                                     " INNER JOIN Comprasnet_VBL.tbl_proposta pp ON pp.prpCod = ppi.prpCod " + 
                                     " INNER JOIN Comprasnet_VBL.tbl_pregaoitem prif ON pri.codmat = prif.codmat and prif.ipgCod != pri.ipgCod " +
                                     " INNER JOIN Comprasnet_VBL.tbl_propostaitem ppif ON prif.ipgCod = ppif.ipgCod and ppif.ippIndAdjudicado = 'S' " +
                                     " INNER JOIN Comprasnet_VBL.tbl_proposta ppf ON ppf.prpCod = ppif.prpCod and pp.prpCNPJ = ppf.prpCNPJ " +
                                    " WHERE pri.prgCod = " + str(prgCod) +
                                    " GROUP BY prif.codmat ")
        #print('frequencias:')
        #print(frequencias)

        #MATERIAL
        materiais = db_siasgnet.query("SELECT distinct ic.numeroitem, ic.codigoitemcatalogo " +
                                    " FROM Siasgnet_dc_Compartilhado_VBL.itemcompra ic " +
                                    " JOIN Siasgnet_dc_Compartilhado_VBL.versaocompraitemcompra vci ON vci.codigoitemcompra = ic.codigoitemcompra " +
                                    " JOIN Siasgnet_dc_Compartilhado_VBL.versaocompra vc ON vc.codigoversaocompra = vci.codigoversaocompra " +
                                    " JOIN Siasgnet_dc_Compartilhado_VBL.compra c ON c.codigocompra = vc.codigocompra " +
                                    " WHERE c.codigomodalidadecompra||c.numerouasgorigem||CAST(c.numerocompra AS INTEGER)||c.anocompra = 5" + str(item['coduasg']) + str(item['numprp']) +
                                    " ORDER BY ic.numeroitem ")
        #print('materiais de:'+ '5' + str(item['coduasg']) + str(item['numprp']))
        #print(materiais)
            
        licitacoes = licitacoes + 1
        
        if (licitacoes % 100) == 0:
            #Atualiza o arquivo a cada 100 pregões processados
            filepath = Path('C:/tmp/brazilian_comprasnet.csv')  
            filepath.parent.mkdir(parents=True, exist_ok=True)  
            dfi.to_csv(filepath) 
    
    # Lance é de empresa punida se estiver entre CNPJs punidos
    if str(item['Competitor']) in punidos:
        dfi.at[index,'Collusive_competitor_original'] = 1
        dfi.at[index,'Collusive_competitor'] = 1

    for lance in lances:
        if lance['ipgCod'] == item['Tender']:
            dfi.at[index,'Number_bids'] = lance['qtdeLances']
            break

    for especificidade in especificidades:
        if especificidade['codmat'] == item['codmat']:
            dfi.at[index,'especificidade'] = especificidade['especificidade']
            break

    for frequencia in frequencias:
        if frequencia['codmat'] == item['codmat']:
            dfi.at[index,'frequencia'] = frequencia['frequencia']
            break

    for material in materiais:
        if material['numeroitem'] == item['ipgItem']:
            dfi.at[index,'material'] = str(material['codigoitemcatalogo'])
            break

    dfi.at[index,'atualizado'] = 1

dfi.info()
    
#Calcula as screens
screens = ['CV', 'SPD', 'DIFFP', 'RD', 'KURT', 'SKEW', 'KSTEST']
dfi = calculate_screen_variables(dfi, screens)
for screen in screens:
    dfi[screen].replace([np.inf, -np.inf], np.nan, inplace=True)
    dfi[screen] = dfi[screen].fillna(0)

dfi.info()

#Atualiza arquivo ao final
filepath = Path(nome_arquivo)  
filepath.parent.mkdir(parents=True, exist_ok=True)  
dfi.to_csv(filepath) 
    
dfi.head(100)

In [ ]:
dfi.info()